In [38]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

In [33]:
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 

# print(f'data: {heart_disease.data}')
# print(f'metadata: {heart_disease.metadata}')
# print(f'var: {heart_disease.variables}')

In [61]:
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets 

y = (y['num'] >  0).astype(int)
# print(y.values)
# print(y.keys())
#features
#1 age
#2 sex
#3 cp
#4 trestbps
#5 chol
#6 fbs
#7 restecg
#8 thalach
#9 exang
#10 oldpeak
#11 slope
#12 ca
#13 thal
print(f'features keys:\n{X.keys()}')
# print('###############################')
# print('###############################')
# print('###############################')
# print(f'Targets num:\n{y}')
# we need to group them "binarize" into 0 & (1,2,3)=1


features keys:
Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal'],
      dtype='object')


In [52]:
#GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [53]:
# Step 0: Normalize data
# z-score all inputs features
cols2zscore = X.keys()
# cols2zscore = cols2zscore.drop('col-name')
X[cols2zscore] = X[cols2zscore].apply(stats.zscore)

C:\Users\elmog\AppData\Local\Temp\ipykernel_10604\1549607059.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cols2zscore] = X[cols2zscore].apply(stats.zscore)


In [54]:
# Step1: Convert numpy data into tensor data
xx = torch.tensor( X[cols2zscore].values ).float()
yy  = torch.tensor( y.values  ).float()

In [55]:
xx.shape

torch.Size([303, 13])

In [56]:
yy.shape

torch.Size([303])

In [57]:
# Step2: use scikit learn to split the data
train_data, test_data, train_labels, test_labels = train_test_split(xx, yy, test_size=0.1)

In [58]:
# Step 3: Convert into PyTorch Datasets
train_data = TensorDataset(train_data, train_labels)
test_data  = TensorDataset(test_data , test_labels)

In [59]:
train_data.tensors[0].shape

torch.Size([272, 13])

In [60]:
test_data.tensors[0].shape

torch.Size([31, 13])

In [62]:
# Step 4: Translate into dataloader objects
batchsize    = 32
train_loader = DataLoader(train_data, batch_size=batchsize, shuffle=True, drop_last=True)
test_loader  = DataLoader(test_data,  batch_size=test_data.tensors[0].shape[0])

In [63]:
# Create the Deep Learning Model
def createTheDiseasePredNet():
    class diseasePredictionNet(nn.Module):
        def __init__(self):
            super().__init__()
            
            self.input = nn.Linear(2,20)
            
            # self.bnormHidden1 =  nn.BatchNorm1d(20)
            self.hidden1 = nn.Linear(20,10)
            
            # self.bnormHidden2 =  nn.BatchNorm1d(10)
            self.hidden2 = nn.Linear(10,5)
            
            self.output = nn.Linear(5,1)
            
        def forward(self, x):
            
            x = F.leaky_relu( self.input(x) )
            
            # x = self.bnormHidden1(x)
            x = F.leaky_relu( self.hidden1(x) )

            # x = self.bnormHidden2(x)
            x = F.leaky_relu( self.hidden2(x) )
                        
            x = self.output(x)
            
            return x
        
    diseasePredictionModel = diseasePredictionNet().to(device)
    
    lossfun = nn.MSELoss()
    
    optimizer = torch.optim.Adam(diseasePredictionModel.parameters(), lr=0.01)#, betas=(0.9, 0.999), eps=1e-8)
    
    return diseasePredictionModel, lossfun, optimizer